In [1]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp
pip install jupyterthemes

  Using cached pip-20.2.4-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=85d44c3acbd18ba204c4036c860020b5598a8927fa8fa9f3417a49bd2d02cfe0
  Stored in directory: /home/ec2-user/.cache/pip/wheels/88/2a/d4/f2e9023989d4d4b3574f268657cb6cd23994665a038803f547
Successfully built smart-open


In [2]:
# create a darker background template (better for my eyes)
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('chesterish')

In [3]:
import time 
import re
import os
import trp
import random 
import boto3
import minecart
import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session
from io import BytesIO

%matplotlib inline

In [4]:
# initiate s3 bucket and corresponding data folder
# bucket = "ran-s3-systemic-risk"
# data_folder ="Input/X-17A-5/"

bucket = "ran-s3-systemic-risk"
data_folder ="FOCUS-OCR/SubsetTest/"

# balance_sheet_idx = 3  # the page of the report you care about

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [5]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['TABLES']
    )
    
    # return response job ID for service
    return response["JobId"]

In [6]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [7]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [8]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, converting tabular data into dataframes**

In [9]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2)
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # itterate through each row within the provided table
    for row in range(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in range(M)]
        
    return pd.DataFrame(arr)

In [10]:
def readPDF(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a pdf
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        # itterate through page tables
        for table in page.tables: 
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty (column 0 = line items)
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
            
            # make sure dataframe is not empty  
            if df.values.size > 0:
                colIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity
                assetCheck = df[colIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity
                debtCheck1 = df[colIndex].str.contains('Liabilities$|^Liabilities', 
                                                      regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[colIndex].str.contains('Liability$|^Liability', 
                                                      regex=True, flags=re.IGNORECASE)

                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty
                check2 = debtCheck1[debtCheck1 == True].empty
                check3 = debtCheck2[debtCheck2 == True].empty

                # if figure matched exactly, we assume this is balance sheet 
                if not check1:
                    catDF.append(df)

                    if check2 == False or check3 == False:  
                        return pd.concat(catDF)
                

## Extract Balance Sheet information

In [11]:
subsetFolder = np.array(session.list_s3_files(bucket, 'Input/SubSets/'))[1:]

In [12]:
subsetFolder

array(['Input/SubSets/1000147-02_subset.pdf',
       'Input/SubSets/1000147-03_subset.pdf',
       'Input/SubSets/1000147-04_subset.pdf', ...,
       'Input/SubSets/99947-17_subset.pdf',
       'Input/SubSets/99947-18_subset.pdf',
       'Input/SubSets/99947-19_subset.pdf'], dtype='<U35')

In [13]:
print('Total files needed to be converted {}'.format(subsetFolder.size))

Total files needed to be converted 78504


In [ ]:
# script to perform OCR (using Textract) for X-17A-5 subsets
outFolder = 'Output/BalanceSheet/'
csvDirectory = np.array(session.list_s3_files(bucket, outFolder))
errorLog = []

# iterate through X-17A-5 subsets stored in s3
for i, key in enumerate(subsetFolder[:10000]):
    # baseFile name (CIK)-{Year}
    baseFile = key.split('/')[-1].split('_')[0]
    fileName = baseFile+'.csv'
    print('\nFile {}'.format(i))
    # if file is not found in directory 
    if outFolder+fileName not in csvDirectory:

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", key)
        
        if res[0]['JobStatus'] != 'FAILED':
            tempDF = readPDF(res)
            print(tempDF)
            # checks for type of return 
            if type(tempDF) == pd.DataFrame:
                # writing data frame to .csv file
                tempDF.to_csv(fileName)

                # the folder used to export to
                xFolder = 'Output/BalanceSheet/'+fileName

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=xFolder, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                print('\tSaved {} file to s3 bucket'.format(baseFile+'.csv'))
            else:
                print('{} no Balance Sheet found'.format(baseFile))
                errorLog.append(baseFile)
        else:
            print('{} could not be parsed'.format(baseFile))
            errorLog.append(baseFile)
    else:
        print('{} has been downloaded'.format(fileName))

print('==========================\nOCR is completed')


File 0
1000147-02.csv has been downloaded

File 1
1000147-03.csv has been downloaded

File 2
1000147-04.csv has been downloaded

File 3
1000147-05.csv has been downloaded

File 4
1000147-06.csv has been downloaded

File 5
1000147-07.csv has been downloaded

File 6
1000147-08.csv has been downloaded

File 7
1000147-09.csv has been downloaded

File 8
1000147-10.csv has been downloaded

File 9
1000147-11.csv has been downloaded

File 10
1000148-02.csv has been downloaded

File 11
1000148-03.csv has been downloaded

File 12
1000148-04.csv has been downloaded

File 13
1000148-05.csv has been downloaded

File 14
1000148-06.csv has been downloaded

File 15
1000148-07.csv has been downloaded

File 16
1000148-08.csv has been downloaded

File 17
1000148-09.csv has been downloaded

File 18
1000148-10.csv has been downloaded

File 19
1000148-11.csv has been downloaded

File 20
1000148-12.csv has been downloaded

File 21
1000148-13.csv has been downloaded

File 22
1000148-14.csv has been downloade

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
None
1001015-13 no Balance Sheet found

File 151
1001015-14.csv has been downloaded

File 152
1001015-15.csv has been downloaded

File 153
1001015-16.csv has been downloaded

File 154
1001015-17.csv has been downloaded

File 155
1001017-02.csv has been downloaded

File 156
1001017-03.csv has been downloaded

File 157
1001017-04.csv has been downloaded

File 158
1001017-05.csv has been downloaded

File 159
1001017-06.csv has been downloaded

File 160
1001017-07.csv has been downloaded

File 161
Started job with id: 9bf98a63167fe3f2ecee12fb50211a1439f16a329aadce7026fcd7b223068302
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
None
1001017-08 no Bala

Started job with id: ccd14533c159100960e5ac61d3fa71d9b6cfdcf2529345f288af6ed59cc43c4b
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
None
1001379-16 no Balance Sheet found

File 306
1001379-17.csv has been downloaded

File 307
1001379-18.csv has been downloaded

File 308
1001379-19.csv has been downloaded

File 309
1001382-02.csv has been downloaded

File 310
1001382-03.csv has been downloaded

File 311
1001382-04.csv has been downloaded

File 312
1001382-05.csv has been downloaded

File 313
1001382-06.csv has been downloaded

File 314
1001382-07.csv has been downloaded

File 315
1001382-08.csv has been downloaded

File 316
1001382-09.csv has been downloaded

File 317
1001382-10.csv has been downloaded

File 318
1001382-11.csv has been downloaded

File 319
1001382-12.csv has been downloaded

File 320
1001382-13.csv has been downloaded

File 321
1001382-15.csv has been downloaded

File 322
1001382-16.csv has been downloaded

File 323
1001382-17.csv has been dow

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
1002134-08 no Balance Sheet found

File 461
1002134-09.csv has been downloaded

File 462
1002134-10.csv has been downloaded

File 463
1002134-11.csv has been downloaded

File 464
1002134-12.csv has been downloaded

File 465
1002134-13.csv has been downloaded

File 466
1002134-14.csv has been downloaded

File 467
1002134-15.csv has been downloaded

File 468
1002134-16.csv has been downloaded

File 469
1002134-17.csv has been downloaded

File 470
Started job with id: f45a3d48e3992a66768786afc26566f030a243614f4c7b92755a02d8df8fb87c
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: FAILED
Resultset page recieved: 1
1002134-18 could not be parsed

File 471
Started job with id: 305f5f8ffb043bfbf0f768054a35bd30873aa93613f9204b22e3b209c45b330d
Job status: IN_PROG

Started job with id: 18e8c58d4d76fc1f580e825fdb7e19b904cf8d439658727ea5fbd458efd7e635
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
1002599-12 no Balance Sheet found

File 588
Started job with id: 41b72735199c71396a69773bb0e77f5e8541eb808d202b418522fd7ba339e927
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
1002599-13 no Balance Sheet found

File 589
1002599-14.csv has been downloaded

File 590
1002599-15.csv has been downloaded

File 591
1002599-16.csv has been downloaded

File 592
1002599-17.csv has been downloaded

File 593
1002599-18.csv has been downloaded

File 594
1002599-19.csv has been downloaded

File 595
1002599-20.csv has been downloaded

File 596
1002600-02.csv has been downloaded

File 597
1002600-03.csv has been downloaded

File 598
1002600-04.csv has been downloaded

File 599
1002600-05.csv has been download

...

	Saved 1013849-07.csv file to s3 bucket

File 2291
Started job with id: 2d9044b9c9565781a4efe5c05cfd1054e09c4e5ae0b5f410e8fdba70f105fae5
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                                   1           2
0                                               Cash    $ 15,848
1                                Accounts receivable      17,082
2                                                       $ 32,930
3               LIABILITIES AND STOCKHOLDER'S EQUITY            
4                        Liability: Accounts payable       $ 112
5  Stockholder's Equity: Common stock, no par val...      20,000
6                                  Retained earnings      24,818
7                 Treasury stock, at cost, 50 share

	Saved 1013850-03.csv file to s3 bucket

File 2295
Started job with id: ff9b110f786f7bedb7287a025024cd78d0312bd859e7af794581156aff1696fb
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    0               1
0                           Cash and cash equivalents           $ 251
1                                        Receivables:                
2                                         Commissions          25 507
3                                     Clearing broker          25 804
4                   Securities owned, at market value          39 064
5   Computer equipment, net of accumulated depreci...             298
6                                        Other assets           3 300
7                                                           $ 114 224
8                LIABILITIES AND SHAREHOLDER'S EQUITY     

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    0          1
0                           Cash and cash equivalents    $ 1,366
1                              Commissions receivable     30,185
2                          Deposit with clearing firm     26,075
3                   Securities owned, at market value      8,811
4                            Due from clearing broker      5,523
5                   Computers and equipment, at cost,           
6           net of accumulated depreciation of $2,424      3,384
7                                        Total assets   $ 75,344
8                LIABILITIES AND SHAREHOLDER'S EQUITY           
9                                        LIABILITIES:           
10                                Commissions payable   $ 45,032
11              Accounts payable and accrued 

Started job with id: ba0d7940b0dbf661454b53c7711db3ec66b2f2ef1b3ffc76ba363ee479697d0a
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    0          1
0                  Cash and cash equivalents (Note 1)  $ 171,776
1                        Deposit with clearing broker     26,074
2                            Due from clearing broker      9,373
3                   Computers and equipment, at cost,           
4           net of accumulated depreciation of $8,279      5,749
5                                        Total assets  $ 212,972
6                LIABILITIES AND SHAREHOLDER'S EQUITY           
7                                        LIABILITIES:           
8         

	Saved 1013850-18.csv file to s3 bucket

File 2310
Started job with id: c9c56942d7706eab4360f2b4ff3451273dd824058791c80bbfcaaeae3280c451
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
                                                    0          1
0                  Cash and cash equivalents (Note 1)   $ 78,383
1                                        Rent deposit      1,780
2                        Deposit with clearing broker     26,075
3                            Due from clearing broker     24,055
4                    Commission receivable non broker    215,650
5                   Computers and equipment, at cost,           
6             net of accumulated depreciation of $423      4,650
7                                        Total assets  $ 350,593
8                LIABILITIES AND SHAREHOLDER'S EQUITY           
9                                  

In [ ]:
np.array(errorLog)

# Helper Functions

In [ ]:
def cleanNumeric(value:str) -> float:
    """
    This function converts a string to a numeric quantity, handles weird string format
    :param: value, string value with hidden numeric quanity  
    :return: floating point values
    
    Complexity -> O(n)
    
    e.g.
        In[0]: $ 19,225     ->   Out[0]: 19255
        In[0]: $ 19,225.76  ->   Out[0]: 19255.76
    """
    # perform regex operation scanning for only numeric quantities/identifiers
    cleanValue = re.sub("[^0-9|.|-]", "", value)
    if cleanValue in ['', '.', '-']:
        return 0.0
    else:
        return float(cleanValue)

In [ ]:
cleanNumeric('-$129.25') + 20